In [1]:
! pip install tensorflow
! pip install h5py
! pip install keras
! pip install nltk
import nltk
nltk.download("all")
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import pandas as pd
import os


  Using cached https://files.pythonhosted.org/packages/35/55/a0dbd642e68e68f3e309d1413abdc0a7aa7e1534c79c0fc2501defb864ac/tensorflow-2.1.0-cp37-cp37m-macosx_10_11_x86_64.whl
     |████████████████████████████████| 1.3MB 6.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/40/23/53ffe290341cd0855d595b0a2e7485932f473798af173bbe3a584b99bb06/tensorboard-2.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/85/7a/ae480be23b768910a9327c33517ced4623ba88dc035f9ce0206657c353a9/scipy-1.4.1-cp37-cp37m-macosx_10_6_intel.whl
  Using cached https://files.pythonhosted.org/packages/c3/fd/1e86bc4837cc9a3a5faf3db9b1854aa04ad35b5f381f9648fbe81a6f94e4/google_pasta-0.1.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/88/97eef84f48fa04fbd6750e62dcceafba6c63c81b7ac1420856c8dcc0a3f9/astor-0.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/18/90/b77c328a1304437ab1310b463e533fa7689f4bfc41549593056d812fab8e/t

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   P

[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package sinica_treebank is already up-to-date!
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /Users/shiqinchoo2019/nltk_data...
[nltk_data]    |   Package smultron is already up-to-date!
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /U

Using TensorFlow backend.


In [3]:
files_list = os.listdir('/Users/shiqinchoo2019/Desktop/ndr_model/ndr/')

In [4]:
path = '/Users/shiqinchoo2019/Desktop/ndr_model/ndr/'

In [5]:
files_names_list = []

for i in range(len(files_list)):
    name = path + files_list[i]
    files_names_list.append(name)

len(files_names_list)

54

In [6]:
texts = []

for i in range(len(files_list)):
    f = open(files_names_list[i], "r", encoding = 'ISO-8859-1')
    text = f.read()
    texts.append(text)

len(texts)

54

In [7]:
files_names_list[1][44:48]
               

'2016'

In [8]:
years = []
for i in range(len(files_names_list)):
    year = files_names_list[i][44:48]
    years.append(year)

years

['2002',
 '2016',
 '2017',
 '2003',
 '2015',
 '2001',
 '2000',
 '2014',
 '2010',
 '2004',
 '2005',
 '2011',
 '2007',
 '2013',
 '2012',
 '2006',
 '1971',
 '1970',
 '1999',
 '1966',
 '1972',
 '1973',
 '1967',
 '1998',
 '1988',
 '1977',
 '1976',
 '1989',
 '1974',
 '1975',
 '1993',
 '1987',
 '1978',
 '1979',
 '1986',
 '1992',
 '1984',
 '1990',
 '1991',
 '1985',
 '1981',
 '1995',
 '1994',
 '1980',
 '1996',
 '1982',
 '1969',
 '1968',
 '1983',
 '1997',
 '2008',
 '2009',
 '2019',
 '2018']

In [9]:
ndr_text = pd.DataFrame()
ndr_text['Year'] = years
ndr_text['Speech'] = texts
ndr_text

,Year,Speech
0,2002,REMAKING SINGAPORE - CHANGING MINDSETS\nI know...
1,2016,"My fellow Singaporeans, good evening again. A ..."
2,2017,"Good evening again. My fellow Singaporeans, we..."
3,2003,\nFROM THE VALLEY TO THE HIGHLANDS\nMy fellow ...
4,2015,"\n My fellow Singaporeans, good evening.\nFir..."
5,2001,NEW SINGAPORE\n\nCLIMBING MOUNT EVEREST\n Whe...
6,2000,TRANSFORMING SINGAPORE\nThese are exciting tim...
7,2014,"âBelieving in Singapore, Pioneering Our Fu..."
8,2010,"\nFellow Singaporeans, good evening.\nEconomy\..."
9,2004,OUR FUTURE OF OPPORTUNITY AND PROMISE\nI Tribu...


In [10]:
all_speech = []
for i in range(len(ndr_text)):
  speech = ndr_text.iloc[i]['Speech']
  all_speech.append(speech)

speech_text = " ".join(all_speech)
len (speech_text)

2592378

In [11]:
speechtext = open("speech_text.txt", "w")
speechtext.write(speech_text)
speechtext.close()

NameError: name 'speech_text' is not defined

In [12]:
import string
 
# turn a doc into clean tokens
def clean_doc(speech_text):
	# replace '--' with a space ' '
	speech_text = speech_text.replace('--', ' ')
	# split into tokens by white space
	tokens = speech_text.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

tokens = clean_doc(speech_text)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))


['remaking', 'singapore', 'changing', 'mindsets', 'i', 'know', 'that', 'many', 'of', 'you', 'are', 'anxious', 'about', 'our', 'economic', 'situation', 'it', 'is', 'not', 'just', 'adult', 'singaporeans', 'our', 'young', 'also', 'seem', 'to', 'be', 'worried', 'in', 'may', 'this', 'year', 'the', 'straits', 'times', 'reported', 'on', 'a', 'dialogue', 'session', 'with', 'junior', 'college', 'polytechnic', 'and', 'university', 'students', 'more', 'than', 'threequarters', 'of', 'the', 'students', 'expressed', 'pessimism', 'about', 'singapores', 'future', 'they', 'were', 'concerned', 'how', 'singapore', 'would', 'fare', 'against', 'giant', 'economies', 'like', 'china', 'and', 'russia', 'and', 'whether', 'one', 'day', 'indonesia', 'and', 'malaysia', 'would', 'overtake', 'us', 'they', 'wondered', 'how', 'many', 'singaporeans', 'would', 'run', 'off', 'to', 'other', 'countries', 'to', 'enjoy', 'cheaper', 'cars', 'and', 'bigger', 'houses', 'i', 'too', 'worry', 'about', 'these', 'things', 'however',

In [13]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
    
# save sequences to file
out_filename = 'ndr_speech.txt'
save_doc(sequences, out_filename)

def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

in_filename = 'ndr_speech.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Total Sequences: 440004
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            885550    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 17711)             1788811   
Total params: 2,825,261
Trainable params: 2,825,261
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='/tmp/ndr.hdf5', verbose=1)
# fit model
model.fit(X, y, batch_size=128, epochs=500, callbacks=[checkpointer])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
  6912/440004 [..............................] - ETA: 51:08 - loss: 8.2398 - accuracy: 0.0402

KeyboardInterrupt: 

In [ ]:
# save the model to file
model.save('ndr.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_ndr.pkl', 'wb'))

In [19]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'ndr_speech.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('ndr.h5')

# load the tokenizer
tokenizer = load(open('tokenizer_ndr.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


not be a problem but for a few that could be a problem and for those who cannot afford the government will subsidise this medishield life premiums for them this is a very very major step we thought about it long and hard we have argued it and we have decided

to make sense for them to help them to help them to him and get more and more choices to our future we will call it a nation to be a cosmopolitan city in the world the government is anxious over the next few years centres we have to foster a large storm of detail to the doldrums and also to tell that the economy is possible for investments have governmenttogovernment nea like thick doors away skills in thailand and the buddhists the christians and the capability criticize us continent for the new hospitals and we will make it into


In [2]:
# load cleaned text sequences
in_filename = 'ndr_speech.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('ndr.h5')

# load the tokenizer
tokenizer = load(open('tokenizer_ndr.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

NameError: name 'load_doc' is not defined

In [22]:
seed_text = "support for elderly"

generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

firms according to efficiency floating suffering from karate view technologies and march in auckland the virtuous ratio gardens meeting in the straits times hokkien the craft i met the grey korean company is called fight with a copy of the future and we have to do it and i think that we should borrow the government will be able to do it and you have to be able to do you understand you and the way we are going to be a big job we are making meritocracy from china and we have to do better and fast polytechnics and
